# Sentiment Analysis using SKLearn

In [1]:
import pandas as pd
import numpy as np

In [2]:
!git clone https://github.com/Rajat6697/IMDB-Movie-Reviews-Large-Dataset-50k-1-47.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k-1-47'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.


In [11]:
df= pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k-1-47/train.xlsx')

# TFIDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [6]:
!pip install git+https://github.com/Rajat6697/preprocess_kgptalkie.git

  Cloning https://github.com/Rajat6697/preprocess_kgptalkie.git to /tmp/pip-req-build-_qjs3doe
  Running command git clone -q https://github.com/Rajat6697/preprocess_kgptalkie.git /tmp/pip-req-build-_qjs3doe
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-cp37-none-any.whl size=11490 sha256=8722977acd725f85275101d4629f7880c2f4452402389ab753170d7932aad10f
  Stored in directory: /tmp/pip-ephem-wheel-cache-98vgsz6v/wheels/fc/13/c8/60304485ffe84b32a577632b814ef5a6005ccf863314351cab
Successfully built preprocess-kgptalkie


In [12]:
import preprocess_kgptalkie as ps
import re

def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [30]:
df['Reviews'] = df['Reviews'].apply(lambda x: get_clean(x))

In [31]:
df.head()

,Reviews,Sentiment
0,when i first tuned in on this morning news i t...,neg
1,mere thoughts of going overboard aka babes aho...,neg
2,why does this movie fall well below standards ...,neg
3,wow and i thought that any steven segal movie ...,neg
4,the story is seen before but that doesand matt...,neg


In [32]:
tfidf= TfidfVectorizer(max_features= 5000)
X= df['Reviews']
y= df['Sentiment']

In [33]:
X= tfidf.fit_transform(X)
X

<25000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2845790 stored elements in Compressed Sparse Row format>

## Train Test Split

In [34]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 42 )

In [35]:
clf= LinearSVC()
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [36]:
y_pred= clf.predict(X_test)

In [37]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         neg       0.88      0.87      0.87      2600
         pos       0.86      0.87      0.87      2400

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000



In [40]:
x_= 'This is a very good movie. Excellent cinematography.'
x= get_clean(x_)
vec = tfidf.transform([x_])

In [42]:
vec.shape

(1, 5000)

In [44]:
clf.predict(vec)

array(['pos'], dtype=object)

In [47]:
import pickle
pickle.dump(clf, open('model_pickle','wb' ))